# Lets-Plot


## Initialization


### Imports


In [ ]:
import os
import re
from typing import Callable

import pandas as pd
import scipy
from lets_plot import *
from lets_plot.mapping import as_discrete
from lets_plot.plot.subplots import SupPlotsSpec

from config.config_type import AllConfig


In [ ]:
LetsPlot.setup_html()

### General Functions


In [ ]:
DataframeItemList = list[tuple[str, pd.DataFrame]]
NamePathList = tuple[list[str], list[str]]
DataSliceDict = dict[str, tuple[int | None, int | None]]
ConfigIndexDict = dict[str, int]

In [ ]:
sparsity_modes = ["point", "grid", "contour", "skeleton", "region"]

In [ ]:
def read_config(ref_path: str, config_index: int = 0) -> AllConfig:
    import json

    filename = f"config_{config_index}.json" if config_index != 0 else "config.json"
    path = os.path.join(os.path.split(ref_path)[0], filename)
    with open(path, "r") as f:
        return json.load(f)

In [ ]:
def get_paths_and_names(
    filter_func: Callable[[str], bool],
    filename: str,
    replace_name_func: Callable[[str], str],
    path: str = "outputs",
) -> NamePathList:
    exp_name_list = list(filter(filter_func, os.listdir(path)))
    path_list = [os.path.join(path, exp_name, filename) for exp_name in exp_name_list]
    name_list = [replace_name_func(exp_name) for exp_name in exp_name_list]
    return path_list, name_list

In [ ]:
def get_dataframes(
    path_list: list[str],
    name_list: list[str],
    data_slice_dict: DataSliceDict | None = None,
    config_index_dict: ConfigIndexDict | None = None,
) -> DataframeItemList:
    df_items = []
    for path, name in zip(path_list, name_list):
        df = pd.read_csv(path)
        if data_slice_dict is not None and data_slice_dict.get(name) is not None:
            df = df.iloc[data_slice_dict[name][0] : data_slice_dict[name][1]]
        if config_index_dict is not None:
            config_index = config_index_dict.get(name, 0)
        else:
            config_index = 0
        config = read_config(path, config_index)
        df["batch_size"] = config["data"]["batch_size"]
        df_items.append((name, df))
    return df_items

In [ ]:
def combine_dataframes(
    df_item_list: DataframeItemList, new_column: str
) -> pd.DataFrame:
    df_list = []
    for name, df in df_item_list:
        new_df = df.copy()
        new_df[new_column] = name
        df_list.append(new_df)
    return pd.concat(df_list)

In [ ]:
def combine_columns(
    df: pd.DataFrame, column_list: list[str], new_column: str
) -> pd.DataFrame:
    df_list = []
    for col in column_list:
        new_df = df.copy()
        new_df.drop(columns=filter(lambda x: x != col, column_list), inplace=True)
        new_df.rename(columns={col: new_column + "_value"}, inplace=True)
        new_df[new_column] = col
        df_list.append(new_df)
    return pd.concat(df_list)

### Data Functions


In [ ]:
def replace_method_name(name: str) -> str:
    return name.replace("WS ", "weasel ").replace("PS ", "protoseg ")


def replace_scenario_name(name: str) -> str:
    return re.sub(f'({"|".join(sparsity_modes)})', "separated", name)

In [ ]:
def get_paths_and_names_with(
    prefix_text: str, suffix_list: list[str] | None, filename: str
) -> tuple[list[str], list[str]]:
    return get_paths_and_names(
        lambda x: x.startswith(prefix_text)
        and ("dummy" not in x)
        and (x.split(" ")[-1] in suffix_list if suffix_list is not None else True),
        filename,
        lambda x: replace_method_name(x.replace(prefix_text + " ", "")),
    )

In [ ]:
def get_meta_dataframe(name_path_list: NamePathList, **kwargs) -> pd.DataFrame:
    path_list, name_list = name_path_list
    df_item_list = get_dataframes(path_list, name_list, **kwargs)
    combined_dataframe = combine_dataframes(df_item_list, "method")
    combined_dataframe["duration"] = combined_dataframe["duration"] / 1000
    combined_dataframe[["method", "scenario"]] = combined_dataframe["method"].str.split(
        " ", expand=True
    )
    return combined_dataframe

In [ ]:
def get_tune_dataframe(name_path_list: NamePathList, **kwargs) -> pd.DataFrame:
    path_list, name_list = name_path_list
    df_item_list = get_dataframes(path_list, name_list, **kwargs)
    dataframe = combine_dataframes(df_item_list, "method")
    dataframe["duration"] = dataframe["duration"] / 1000
    dataframe[["method", "scenario"]] = dataframe["method"].str.split(" ", expand=True)
    dataframe["scenario"] = dataframe["scenario"].apply(replace_scenario_name)
    dataframe["sparsity"] = (
        dataframe["sparsity_mode"] + "=" + dataframe["sparsity_value"].astype(str)
    )
    return dataframe

In [ ]:
def get_weasel_tune_dataframe(name_path_list: NamePathList, **kwargs) -> pd.DataFrame:
    path_list, name_list = name_path_list
    df_item_list = get_dataframes(path_list, name_list, **kwargs)
    dataframe = combine_dataframes(df_item_list, "scenario")
    dataframe["test_duration"] = dataframe["test_duration"] / 1000
    dataframe["scenario"] = dataframe["scenario"].apply(replace_scenario_name)
    dataframe["sparsity"] = (
        dataframe["sparsity_mode"] + "=" + dataframe["sparsity_value"].astype(str)
    )
    return dataframe

### Chart Functions


In [ ]:
def plot_meta_loss(dataframe: pd.DataFrame) -> ggplot:
    num_methods = len(dataframe["method"].unique())
    return (
        ggplot(dataframe)
        + geom_line(aes(x="epoch", y="loss", color="scenario"))
        + facet_grid(y="method", scales="free_y", y_order=0)
        + ggsize(1200, 400 * num_methods)
        + flavor_darcula()
    )

In [ ]:
def plot_meta_loss_multiple(dataframe: pd.DataFrame) -> ggplot:
    num_methods = len(dataframe["method"].unique())
    num_scenarios = len(dataframe["scenario"].unique())
    return (
        ggplot(dataframe)
        + geom_line(aes(x="epoch", y="loss", color="scenario"))
        + facet_grid(x="method", y="scenario", scales="free", x_order=0, y_order=0)
        + ggsize(600 * num_methods, 200 * num_scenarios + 50)
        + flavor_darcula()
        + theme(panel_border=element_rect(size=1), legend_position="bottom")
    )

In [ ]:
def plot_meta_metric(dataframe: pd.DataFrame, size_col: str = "") -> ggplot:
    new_dataframe = combine_columns(
        dataframe, ["duration", "post_gpu_percent"], "metric"
    )
    num_methods = len(new_dataframe["method"].unique())
    return (
        ggplot(new_dataframe)
        + geom_line(
            aes(
                x="epoch",
                y="metric_value",
                color="scenario",
                size=as_discrete(size_col, order=1) if size_col != "" else None,
            )
        )
        + facet_grid(x="method", y="metric", scales="free_y", x_order=0, y_order=0)
        + ggsize(600 * num_methods, 800)
        + flavor_darcula()
        + scale_size(range=[0.5, 1])
        + theme(panel_border=element_rect(size=1), legend_position="bottom")
    )

In [ ]:
def plot_tune_score_by_scenario(dataframe: pd.DataFrame, size_col: str = "") -> ggplot:
    new_dataframe = combine_columns(dataframe, ["iou_od", "iou_oc"], "iou")
    num_sparsity = len(new_dataframe["sparsity"].unique())
    return (
        ggplot(new_dataframe)
        + geom_line(
            aes(
                x=as_discrete("epoch"),
                y="iou_value",
                color="scenario",
                linetype="method",
                size=as_discrete(size_col, order=1) if size_col != "" else None,
            ),
            tooltips=layer_tooltips().line("@method @scenario").line("@iou_value"),
        )
        + facet_grid(x="sparsity", y="iou", scales="fixed", x_order=1, y_order=0)
        + ggsize(200 * num_sparsity, 600)
        + flavor_darcula()
        + scale_size(range=[0.5, 1])
        + theme(panel_border=element_rect(size=1), legend_position="top")
    )

In [ ]:
def plot_tune_metric_by_scenario(dataframe: pd.DataFrame, size_col: str = "") -> ggplot:
    new_dataframe = combine_columns(
        dataframe, ["duration", "post_gpu_percent"], "metric"
    )
    num_sparsity = len(new_dataframe["sparsity"].unique())
    return (
        ggplot(new_dataframe)
        + geom_line(
            aes(
                x=as_discrete("epoch"),
                y="metric_value",
                color="scenario",
                linetype="method",
                size=as_discrete(size_col, order=1) if size_col != "" else None,
            ),
            tooltips=layer_tooltips()
            .line("@method @scenario")
            .format("@metric_value", ".2f")
            .line("@metric_value"),
        )
        + facet_grid(x="sparsity", y="metric", scales="free_y", x_order=1, y_order=0)
        + ggsize(200 * num_sparsity, 600)
        + flavor_darcula()
        + scale_size(range=[0.5, 1])
        + theme(panel_border=element_rect(size=1), legend_position="top")
    )

In [ ]:
def plot_weasel_tune_score_by_scenario(
    dataframe: pd.DataFrame, value_col: str, size_col: str = ""
) -> ggplot:
    num_epoch = len(dataframe["epoch"].unique())
    num_sparsity = len(dataframe["sparsity"].unique())
    return (
        ggplot(dataframe)
        + geom_line(
            aes(
                x=as_discrete("tune_epoch"),
                y=value_col,
                color="scenario",
                size=as_discrete(size_col, order=1) if size_col != "" else None,
            )
        )
        + facet_grid(x="epoch", y="sparsity", scales="fixed", x_order=1, y_order=1)
        + ggsize(250 * num_epoch, 200 * num_sparsity + 50)
        + flavor_darcula()
        + scale_size(range=[0.5, 1])
        + theme(panel_border=element_rect(size=1), legend_position="top")
    )

In [ ]:
def plot_tune_score_by_sparsity(dataframe: pd.DataFrame) -> SupPlotsSpec:
    new_dataframe = combine_columns(dataframe, ["iou_od", "iou_oc"], "iou")
    plot_list = []
    sorted_sparsity_modes = sorted(new_dataframe["sparsity_mode"].unique())
    num_sparsity = len(sorted_sparsity_modes)
    for index, sparsity_mode in enumerate(sorted_sparsity_modes):
        sparsity_plot = (
            ggplot(new_dataframe[new_dataframe["sparsity_mode"] == sparsity_mode])
            + geom_line(
                aes(
                    x="sparsity_value",
                    y="iou_value",
                    color="method",
                    linetype=as_discrete("epoch", order=-1),
                ),
                tooltips=layer_tooltips().line("@method ep=@epoch").line("@iou_value"),
            )
            + ggtitle(sparsity_mode)
            + facet_grid(x="n_shots", y="iou", scales="free_x", x_order=0, y_order=0)
            + ylim(0, 1)
            + flavor_darcula()
            + theme(panel_border=element_rect(size=1), title=element_text(hjust=0.5))
        )
        if (
            index == num_sparsity - 1
            or index == num_sparsity - 2
            and num_sparsity % 2 == 0
        ):
            sparsity_plot += theme(legend_position="bottom")
        else:
            sparsity_plot += theme(legend_position="none")
        plot_list.append(sparsity_plot)
    num_rows = num_sparsity // 2 + num_sparsity % 2
    row_relative_heights = [1 for _ in range(num_rows - 1)] + [1.25]
    return (
        gggrid(plot_list, ncol=2, vspace=30, heights=row_relative_heights)
        + ggsize(1200, 400 * num_rows)
        + flavor_darcula()
    )

In [ ]:
def plot_tune_metric_by_sparsity(dataframe: pd.DataFrame, size_col: str = "") -> ggplot:
    new_dataframe = combine_columns(
        dataframe, ["duration", "post_gpu_percent"], "metric"
    )
    new_dataframe["method__sparsity_value"] = (
        new_dataframe["method"] + " s=" + new_dataframe["sparsity_value"].astype(str)
    )
    num_sparsity = len(new_dataframe["sparsity_mode"].unique())
    return (
        ggplot(new_dataframe)
        + geom_line(
            aes(
                x="n_shots",
                y="metric_value",
                color="method__sparsity_value",
                linetype=as_discrete("epoch", order=-1),
                size=as_discrete(size_col, order=1) if size_col != "" else None,
            ),
            tooltips=layer_tooltips()
            .line("@method ep=@epoch s=@sparsity_value")
            .format("@metric_value", ".2f")
            .line("@metric_value"),
        )
        + facet_grid(
            x="sparsity_mode", y="metric", scales="free_y", x_order=1, y_order=0
        )
        + ggsize(200 * num_sparsity, 700)
        + flavor_darcula()
        + scale_size(range=[0.5, 1])
        + scale_color_discrete(guide=guide_legend(ncol=6))
        + theme(panel_border=element_rect(size=1), legend_position="top")
    )

## Exp - Scenario


In [ ]:
prefix = "v3 RO-DR L"
suffixes = ["all"] + sparsity_modes + list(map(lambda x: x + "-var", sparsity_modes))
slice_dict = {"weasel all": (0, 70), "protoseg all": (0, 25)}

### Meta


In [ ]:
paths, names = get_paths_and_names_with(prefix, suffixes, "train_loss.csv")
combined_df = get_meta_dataframe((paths, names))

print(names)
combined_df

In [ ]:
plot_meta_loss(combined_df)

-   "weasel" method is less stable than "protoseg" method, but better at optimizing "all" scenarios and others.
-   "protoseg" method has trouble optimizing "all" scenarios and "contour" scenario.
-   "all" scenarios is less stable than "separated" scenarios, especially for "protoseg" method.
-   "all-more-embeds" scenario for "protoseg" method is not significantly different than "all" scenario.


In [ ]:
plot_meta_metric(combined_df, "batch_size")

-   "weasel" method is slower than "protoseg" method, but the GPU usage is little bit lower, note that "protoseg" method has a much higher batch size.
-   Order of scenarios from slowest to fastest: "region" > "all" = "skeleton" > "contour" > "point" > "grid".
-   Note that "all" scenarios include "point_old" and "grid_old".
-   "region" scenario is significantly slower than others.
-   "all-more-embeds" scenario use same amount of time, but higher GPU usage than the "all" scenario.
-   "point" scenarios has significantly higher GPU usage than others.


### Tune


In [ ]:
paths, names = get_paths_and_names_with(prefix, suffixes, "tuned_score.csv")
combined_df = get_tune_dataframe((paths, names), data_slice_dict=slice_dict)
combined_df_full = combined_df.copy()
# combined_df = combined_df[combined_df['n_shots'] == 10]

print(names)
combined_df

In [ ]:
plot_tune_score_by_scenario(combined_df, "n_shots")

In [ ]:
combined_df_flat = combine_columns(combined_df, ["iou_od", "iou_oc"], "iou")
combined_df_flat = combined_df_flat[combined_df_flat["n_shots"] == 10]
point_grid_df_flat = combined_df_flat[combined_df_flat["scenario"] == "all"]
point_grid_df_flat = point_grid_df_flat[
    point_grid_df_flat["sparsity_mode"].isin(["point", "grid", "point_old", "grid_old"])
]
point_grid_df_flat["sparsity_newness"] = point_grid_df_flat["sparsity_mode"].apply(
    lambda x: "new" if "old" not in x else "old"
)
point_grid_df_flat["sparsity_mode"] = point_grid_df_flat["sparsity_mode"].apply(
    lambda x: x.replace("_old", "")
)
point_grid_df_flat["method__sparsity_mode"] = (
    point_grid_df_flat["method"] + " " + point_grid_df_flat["sparsity_mode"]
)

(
    ggplot(point_grid_df_flat)
    + geom_line(aes(x=as_discrete("epoch"), y="iou_value", color="sparsity_newness"))
    + facet_grid(
        x="method__sparsity_mode", y="iou", scales="fixed", x_order=0, y_order=0
    )
    + ggsize(1200, 600)
    + flavor_darcula()
    + theme(panel_border=element_rect(size=1), legend_position="top")
)

In [ ]:
plot_tune_metric_by_scenario(combined_df_full, "n_shots")

### Weasel Tune


In [ ]:
paths, names = get_paths_and_names_with("v3 RO-DR L WS", suffixes, "tuning_score.csv")

combined_df = get_weasel_tune_dataframe(
    (paths, names), data_slice_dict={"all": (0, 70)}
)
combined_df = combined_df[combined_df["n_shots"] == 10]

print(names)
combined_df

In [ ]:
plot_weasel_tune_score_by_scenario(combined_df, "iou_oc")

## Exp - Loss - Short


In [ ]:
prefix = "v3 RO-DR S"
suffixes = ["all", "all-bce", "all-bce_2", "all-iou", "all-iou_bce"]

### Meta


In [ ]:
paths, names = get_paths_and_names_with(prefix, suffixes, "train_loss.csv")
combined_df = get_meta_dataframe((paths, names))

print(names)
combined_df

In [ ]:
plot_meta_loss_multiple(combined_df)

In [ ]:
plot_meta_metric(combined_df)

### Tune


In [ ]:
paths, names = get_paths_and_names_with(prefix, suffixes, "tuned_score.csv")
combined_df = get_tune_dataframe((paths, names))

print(names)
combined_df

In [ ]:
plot_tune_score_by_scenario(combined_df)

In [ ]:
plot_tune_metric_by_scenario(combined_df)

### Weasel Tune


In [ ]:
paths, names = get_paths_and_names_with("v3 RO-DR S WS", suffixes, "tuning_score.csv")
combined_df = get_weasel_tune_dataframe((paths, names))

print(names)
combined_df

In [ ]:
plot_weasel_tune_score_by_scenario(combined_df, "iou_od")

## Exp - Loss - Long


In [ ]:
prefix = "v3 RO-DR L"
suffixes = ["all", "all-iou", "all-poor"]
slice_dict = {"weasel all": (0, 70), "protoseg all": (0, 25)}

### Meta


In [ ]:
paths, names = get_paths_and_names_with(prefix, suffixes, "train_loss.csv")
combined_df = get_meta_dataframe((paths, names))

print(names)
combined_df

In [ ]:
plot_meta_loss_multiple(combined_df)

In [ ]:
plot_meta_metric(combined_df, "batch_size")

### Tune


In [ ]:
paths, names = get_paths_and_names_with(prefix, suffixes, "tuned_score.csv")
combined_df = get_tune_dataframe((paths, names), data_slice_dict=slice_dict)

print(names)
combined_df

In [ ]:
plot_tune_score_by_scenario(combined_df, "n_shots")

In [ ]:
plot_tune_metric_by_scenario(combined_df, "n_shots")

### Weasel Tune


In [ ]:
paths, names = get_paths_and_names_with("v3 RO-DR L WS", suffixes, "tuning_score.csv")
combined_df = get_weasel_tune_dataframe(
    (paths, names), data_slice_dict={"all": (0, 350)}
)

print(names)
combined_df

In [ ]:
plot_weasel_tune_score_by_scenario(combined_df, "iou_od", "n_shots")

## Exp - Sparsity Value


In [ ]:
prefix = "v3 RO-DR L"
suffixes = ["all"]
slice_dict = {"weasel all": (70, None), "protoseg all": (25, None)}

### Tune


In [ ]:
paths, names = get_paths_and_names_with(prefix, suffixes, "tuned_score.csv")
combined_df = get_tune_dataframe((paths, names), data_slice_dict=slice_dict)

print(names)
combined_df

In [ ]:
plot_tune_score_by_sparsity(
    combined_df[(combined_df["method"] == "weasel") & (combined_df["epoch"] == 200)]
)

In [ ]:
plot_tune_metric_by_sparsity(combined_df, "batch_size")

### Tune Distribution


In [ ]:
iou_od_low, iou_oc_low = 0.8647, 0.5202
iou_od_high, iou_oc_high = 0.9386, 0.8382

In [ ]:
combined_df_2 = combine_columns(combined_df, ["iou_od", "iou_oc"], "iou")
combined_df_2["iou_low"] = combined_df_2.apply(
    lambda x: iou_od_low if x["iou"] == "iou_od" else iou_oc_low, axis=1
)
combined_df_2["iou_high"] = combined_df_2.apply(
    lambda x: iou_od_high if x["iou"] == "iou_od" else iou_oc_high, axis=1
)

In [ ]:
(
    ggplot(combined_df_2[combined_df_2["sparsity_mode"] != "skeleton"])
    + geom_density(
        aes(x="iou_value", group="method", color="method", fill="method"), alpha=0.3
    )
    + geom_vline(aes(xintercept="iou_low"), size=0.5, linetype="longdash")
    + geom_vline(aes(xintercept="iou_high"), size=0.5, linetype="longdash")
    + facet_grid(x="iou", scales="fixed")
    + ggsize(1200, 400)
    + flavor_darcula()
    + theme(legend_position="top")
)

In [ ]:
(
    ggplot(combined_df_2)
    + geom_density(
        aes(x="iou_value", group="method", color="method", fill="method"), alpha=0.3
    )
    + geom_vline(aes(xintercept="iou_low"), size=0.5, linetype="longdash")
    + geom_vline(aes(xintercept="iou_high"), size=0.5, linetype="longdash")
    + facet_grid(x="iou", y="sparsity_mode", scales="fixed")
    + ggsize(1200, 1400)
    + flavor_darcula()
    + theme(legend_position="top")
)

In [ ]:
alpha = 0.05
get_t_value = lambda n: scipy.stats.t.ppf(1 - alpha / 2, n - 1)

combined_df_3 = combined_df.groupby(["method", "sparsity_mode"])[
    ["iou_od", "iou_oc"]
].agg(
    count=("iou_od", "count"),
    iou_od_mean=("iou_od", "mean"),
    iou_oc_mean=("iou_oc", "mean"),
    iou_od_std=("iou_od", "std"),
    iou_oc_std=("iou_oc", "std"),
)

combined_df_3_od = combined_df_3.drop(columns=["iou_oc_mean", "iou_oc_std"])
combined_df_3_od = combined_df_3_od.rename(
    columns={"iou_od_mean": "mean_value", "iou_od_std": "std_value"}
)
combined_df_3_od["iou"] = "iou_od"
combined_df_3_oc = combined_df_3.drop(columns=["iou_od_mean", "iou_od_std"])
combined_df_3_oc = combined_df_3_oc.rename(
    columns={"iou_oc_mean": "mean_value", "iou_oc_std": "std_value"}
)
combined_df_3_oc["iou"] = "iou_oc"
combined_df_3 = pd.concat([combined_df_3_od, combined_df_3_oc])

combined_df_3["iou_low"] = combined_df_3.apply(
    lambda x: iou_od_low if x["iou"] == "iou_od" else iou_oc_low, axis=1
)
combined_df_3["iou_high"] = combined_df_3.apply(
    lambda x: iou_od_high if x["iou"] == "iou_od" else iou_oc_high, axis=1
)

combined_df_3["mean_error"] = combined_df_3.apply(
    lambda x: get_t_value(x["count"]) * x["std_value"] / x["count"] ** 0.5, axis=1
)
combined_df_3["mean_low_limit"] = (
    combined_df_3["mean_value"] - combined_df_3["mean_error"]
)
combined_df_3["mean_high_limit"] = (
    combined_df_3["mean_value"] + combined_df_3["mean_error"]
)

combined_df_3.reset_index(inplace=True)
combined_df_3

In [ ]:
(
    ggplot(combined_df_3)
    + geom_pointrange(
        aes(
            x="sparsity_mode",
            y="mean_value",
            ymin="mean_low_limit",
            ymax="mean_high_limit",
            color="sparsity_mode",
        )
    )
    + geom_hline(aes(yintercept="iou_low"), size=0.5, linetype="longdash")
    + geom_hline(aes(yintercept="iou_high"), size=0.5, linetype="longdash")
    + facet_grid(x="method", y="iou", scales="fixed")
    + ggsize(1200, 800)
    + flavor_darcula()
)

## Exploration - All Exp


In [ ]:
prefix = "v3 RO-DR L"
suffixes = None

In [ ]:
paths, names = get_paths_and_names_with(prefix, suffixes, "tuned_score.csv")
combined_df = get_tune_dataframe((paths, names))

print(names)
combined_df

In [ ]:
# threshold_od, threshold_oc = 0.94, 0.84
threshold_od, threshold_oc = 0.9, 0.8
score_good_df = combined_df[
    (combined_df["iou_od"] > threshold_od) & (combined_df["iou_oc"] > threshold_oc)
]

print(score_good_df["iou_od"].mean(), score_good_df["iou_oc"].mean())

score_good_df.groupby(["method", "sparsity_mode"])[["iou_od", "iou_oc"]].agg(
    count=("iou_od", "count"),
    iou_od_mean=("iou_od", "mean"),
    iou_oc_mean=("iou_oc", "mean"),
)

In [ ]:
scenario_all_df = combined_df[combined_df["scenario"] == "all"]
scenario_all_df = scenario_all_df[
    (scenario_all_df["iou_od"] > 0.5) & (scenario_all_df["iou_oc"] > 0.5)
]

scenario_all_df.groupby(["method", "sparsity_mode"])[["iou_od", "iou_oc"]].agg(
    count=("iou_od", "count"),
    iou_od_mean=("iou_od", "mean"),
    iou_oc_mean=("iou_oc", "mean"),
)

In [ ]:
scenario_all_df = combined_df[combined_df["scenario"] == "all"]

scenario_all_df.groupby(["method", "epoch"])[["iou_od", "iou_oc"]].agg(
    count=("iou_od", "count"),
    iou_od_mean=("iou_od", "mean"),
    iou_oc_mean=("iou_oc", "mean"),
)

## Exploration - Plot


In [ ]:
weasel_train_loss = pd.read_csv("outputs/v1 RO-DR L WS/train_loss.csv")
protoseg_train_loss = pd.read_csv("outputs/v1 RO-DR L PS/train_loss.csv")

train_loss = combine_dataframes(
    [("weasel", weasel_train_loss), ("protoseg", protoseg_train_loss)], "method"
)

In [ ]:
(
    ggplot(train_loss)
    + geom_line(aes(x="epoch", y="duration", color="method"))
    + facet_grid("method", scales="free")
    + ggsize(800, 300)
    + flavor_darcula()
)

In [ ]:
(
    ggplot(train_loss)
    + geom_line(aes(x="epoch", y="duration", color="method"))
    + facet_grid("method", scales="free")
    + ggsize(800, 300)
    + flavor_darcula()
    + theme(
        legend_position="left",
        line=element_line(size=2),
        # rect=element_rect(size=10),
        text=element_text(size=10),
        axis_text=element_text(size=5),
        axis_title=element_text(size=20),
        axis_line_y=element_line(size=2),
        panel_border=element_rect(size=2, color="yellow"),
        plot_margin=margin(0.5, 0.5, 0.5, 0.5),
        strip_text="blank",
    )
)

# Matplotlib


## Initialization


### Imports


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use("dark_background")


## Meta


### Loss & Duration


In [ ]:
weasel_train_loss = pd.read_csv("outputs/v1 RO-DR L WS/train_loss.csv")
protoseg_train_loss = pd.read_csv("outputs/v1 RO-DR L WS/train_loss.csv")

_, axs = plt.subplots(2, 2, figsize=(14, 10))

axs[0][0].set_title("weasel - epoch vs loss")
axs[0][0].plot(weasel_train_loss["epoch"], weasel_train_loss["loss"])

axs[0][1].set_title("protoseg - epoch vs loss")
axs[0][1].plot(protoseg_train_loss["epoch"], protoseg_train_loss["loss"])

axs[1][0].set_title("epoch vs duration in ms")
axs[1][0].plot(
    weasel_train_loss["epoch"], weasel_train_loss["duration"], label="weasel"
)
axs[1][0].plot(
    protoseg_train_loss["epoch"], protoseg_train_loss["duration"], label="protoseg"
)
axs[1][0].legend()

## Tune


### Score


In [ ]:
weasel_tuned_score = pd.read_csv("outputs/v1 RO-DR L WS/tuned_score.csv")
protoseg_tuned_score = pd.read_csv("outputs/v1 RO-DR L WS/tuned_score.csv")

sparsity_modes = ["point", "grid", "contour", "skeleton", "region", "dense"]

_, axs = plt.subplots(3, 2, figsize=(12, 15))

for sm in sparsity_modes:
    weasel_df = weasel_tuned_score[weasel_tuned_score["sparsity_mode"] == sm]
    weasel_epochs = [str(ep) for ep in weasel_df["epoch"]]
    axs[0][0].plot(weasel_epochs, weasel_df["iou_od"], label=sm)
    axs[1][0].plot(weasel_epochs, weasel_df["iou_oc"], label=sm)
    axs[2][0].plot(weasel_epochs, weasel_df["duration"], label=sm)

    protoseg_df = protoseg_tuned_score[protoseg_tuned_score["sparsity_mode"] == sm]
    protoseg_epochs = [str(ep) for ep in protoseg_df["epoch"]]
    axs[0][1].plot(protoseg_epochs, protoseg_df["iou_od"], label=sm)
    axs[1][1].plot(protoseg_epochs, protoseg_df["iou_oc"], label=sm)
    axs[2][1].plot(protoseg_epochs, protoseg_df["duration"], label=sm)

axs[0][0].set_title("weasel - epoch vs optic disc IoU")
axs[1][0].set_title("weasel - epoch vs optic cup IoU")
axs[2][0].set_title("weasel - epoch vs duration")
axs[0][0].legend()
axs[1][0].legend()
axs[2][0].legend()
axs[0][0].set_ylim([0, 1])
axs[1][0].set_ylim([0, 1])
axs[0][1].set_title("protoseg - epoch vs optic disc IoU")
axs[1][1].set_title("protoseg - epoch vs optic cup IoU")
axs[2][1].set_title("protoseg - epoch vs duration")
axs[0][1].legend()
axs[1][1].legend()
axs[2][1].legend()
axs[0][1].set_ylim([0, 1])
axs[1][1].set_ylim([0, 1])

## Weasel Tune


### Score


In [ ]:
weasel_tuning_score = pd.read_csv("outputs/v1 RO-DR L WS/tuning_score.csv")
weasel_epochs = weasel_tuning_score["epoch"].unique()

sparsity_modes = ["point", "grid", "contour", "skeleton", "region", "dense"]

_, axs = plt.subplots(len(weasel_epochs), 2, figsize=(12, 5 * len(weasel_epochs)))

for i, ep in enumerate(weasel_epochs):
    for sm in sparsity_modes:
        weasel_df = weasel_tuning_score[
            (weasel_tuning_score["sparsity_mode"] == sm)
            & (weasel_tuning_score["epoch"] == ep)
        ]
        weasel_tune_epochs = [str(tep) for tep in weasel_df["tune_epoch"]]
        axs[i][0].plot(weasel_tune_epochs, weasel_df["iou_od"], label=sm)
        axs[i][1].plot(weasel_tune_epochs, weasel_df["iou_oc"], label=sm)
        # axs[i][2].plot(weasel_tune_epochs, weasel_df['test_duration'], label = sm)

    axs[i][0].set_title(f"weasel ep-{ep} - tune epoch vs optic disc IoU")
    axs[i][1].set_title(f"weasel ep-{ep} - tune epoch vs optic cup IoU")
    # axs[i][2].set_title(f'weasel ep-{ep} - tune epoch vs test duration')
    axs[i][0].legend()
    axs[i][1].legend()
    # axs[i][2].legend()
    axs[i][0].set_ylim([0, 1])
    axs[i][1].set_ylim([0, 1])

# Other
